In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb
import unicodedata
import importlib
import numpy as np
import csv

from pathlib import Path
import re
import pandas as pd

import instances_embeddings_eval

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

2025-12-23 23:13:34.593937: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 23:13:34.650464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jupyter-vojta/notebooks/labyrinth/venv_torch_nlp/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
import grela_helpers

In [3]:
conn = grela_helpers.make_connection()

In [4]:
#grela_extraction_helpers.conn.close()
# importlib.reload(grela_extraction_helpers)

In [5]:
# establish connection with Google sheets
file_data = json.load(open(os.path.expanduser("../../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ruland_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1od248fJpNbwMO8IVZQVO3xnzrmmTe68yS9XWajyuk5o/edit?usp=sharing")

In [6]:
lexeme_df = pd.read_parquet("../data/ruland-dictionaries.parquet")

In [7]:
lexeme_df["target_len"] = lexeme_df["target_canonical"].str.split().str.len()

In [8]:
lexeme_df = lexeme_df[lexeme_df["target_relemmatized"].str.split().str.len() < 4]

In [9]:
MAX_N = 3
lexeme_df = lexeme_df[lexeme_df["target_len"] <= MAX_N].copy()
len(lexeme_df)

2846

In [10]:
def safe_name(s: str | None) -> str:
    """Make a safe short filename component from a target phrase."""
    if not s or not isinstance(s, str):
        return "unknown"
    s = s.strip().lower().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]+", "", s)   # keep only safe chars
    return s or "unnamed"

In [11]:
lexeme_df["instance_fname"] = lexeme_df["target_canonical"].apply(lambda x: safe_name(x) + ".parquet")

In [12]:
outdir = Path("../data/large_files/emlap_ruland_instances/")
outdir.mkdir(exist_ok=True)

In [13]:
def read_hits(fname):
    try:
        path = os.path.join(outdir, fname)
        instances = pd.read_parquet(path).to_dict("records")
    except:
        instances = []
    return instances

In [14]:
df = pd.DataFrame(read_hits("mercurius_metallorum.parquet"))

In [22]:
df = pd.DataFrame(read_hits("aurum.parquet"))

In [23]:
noscemus_emlap_ids = ['713324', '26765', '597737', '971293', '688159']

In [24]:
noscemus_alchemy_ids = pd.read_pickle("../data/noscemus_alchemy_ids.pickle")
noscemus_alchemy_ids[:10]

9979      noscemus_705665
9991      noscemus_658534
10005     noscemus_863735
10025     noscemus_713324
10029    noscemus_1031760
10052     noscemus_797710
10059     noscemus_664562
10060     noscemus_605035
10061     noscemus_694478
10085     noscemus_801655
Name: grela_id, dtype: object

In [25]:
noscemus_ruland = ['668514']

In [26]:
to_exclude = ["noscemus_" + id for id in noscemus_emlap_ids + noscemus_ruland]
to_exclude

['noscemus_713324',
 'noscemus_26765',
 'noscemus_597737',
 'noscemus_971293',
 'noscemus_688159',
 'noscemus_668514']

In [27]:
len(df)

69768

In [28]:
# after filtering out all Ruland itself and noscemus in emlap overlap
df = df[df["grela_id"].apply(lambda x: x not in to_exclude)]
len(df)

68892

In [29]:
def get_subset(grela_id):
    grela_source = grela_id.partition("_")[0]
    if grela_source == "noscemus":
        if grela_id in noscemus_alchemy_ids.tolist():
            return "noscemus_alchymia"
        else:
            return "noscemus_rest"
    else:
        return grela_source

In [30]:
#df["grela_source"] = df["grela_id"].apply(lambda x: x.partition("_")[0])

In [31]:
df["subsets"] = df["grela_id"].apply(lambda x: get_subset(x))

In [32]:
df.value_counts("subsets").to_dict()

{'cc': 37216,
 'noscemus_rest': 14074,
 'emlap': 11585,
 'noscemus_alchymia': 5701,
 'vulgate': 316}

In [33]:
api_url = "https://ccs-lab.zcu.cz/grela-api/api/query"
query = "SELECT * FROM works"
works_df = pd.read_parquet(requests.post(api_url, json={"query": query}).json()["download_url"])

In [34]:
import json
import pandas as pd

def to_dict(x):
    if pd.isna(x):
        return {}
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)   # -> dict
        except json.JSONDecodeError:
            return {}
    return {}

works_df["noscemus_discipline"] = (
    works_df["subcorpus_specific_metadata"]
      .map(to_dict)
      .map(lambda d: d.get("noscemus_discipline"))
)

In [35]:
works_df["subset"] = works_df["grela_id"].apply(get_subset)

In [36]:
works_df.value_counts("subset")

subset
cc                   7819
lagt                 2160
noscemus_rest         872
noscemus_alchymia     103
emlap                 100
vulgate                73
Name: count, dtype: int64

In [37]:
corpora_token_counts_dict = works_df.groupby("subset").sum()["token_count"].to_dict()
corpora_token_counts_dict

{'cc': 254770887,
 'emlap': 6477016,
 'lagt': 38223149,
 'noscemus_alchymia': 14351698,
 'noscemus_rest': 122973685,
 'vulgate': 603091}

In [38]:
corpora_token_counts_dict['grela_la'] = sum([corpora_token_counts_dict["cc"], corpora_token_counts_dict["noscemus_alchymia"], corpora_token_counts_dict["noscemus_rest"],corpora_token_counts_dict["emlap"]])
corpora_token_counts_dict

{'cc': 254770887,
 'emlap': 6477016,
 'lagt': 38223149,
 'noscemus_alchymia': 14351698,
 'noscemus_rest': 122973685,
 'vulgate': 603091,
 'grela_la': 398573286}

In [39]:
import instances_embeddings_eval

In [67]:
def _is_missing(v) -> bool:
    # safe missing check for scalars; won't choke on list/dict
    return v is None or v is pd.NA or (isinstance(v, float) and np.isnan(v))


def obtain_overview_data(fname):
    try:
        df = pd.DataFrame(read_hits(fname))
        df = df[df["grela_id"].apply(lambda x: x not in to_exclude)] # filter out ruland himself and noscemus included in emlap
        df["subsets"] = df["grela_id"].apply(lambda x: get_subset(x))
        subsets = ["emlap", "noscemus_alchymia", "noscemus_rest", "cc"]
        df = df[df["subsets"].isin(subsets)]
        df_value_counts = df.value_counts("subsets").to_dict()
        df_value_counts["grela_la"] = len(df)
        overview_data = {}
        for subset in subsets + ["grela_la"]:
            overview_data[subset + "_instances_N"] = df_value_counts[subset]
        for subset in subsets + ["grela_la"]:
            overview_data[subset + "_instances_freq"] = df_value_counts[subset] / corpora_token_counts_dict[subset]
        overview_data["emlap_vs_grela_la_rel_freq"] = overview_data["emlap_instances_freq"] / overview_data["grela_la_instances_freq"]
        instances_sample, instances_samples_sizes_dict = instances_embeddings_eval.produce_and_enrich_samples(df)
        order = instances_embeddings_eval.present_order(instances_sample, instances_embeddings_eval.palette)
        S, Ps = instances_embeddings_eval.avg_cosine_matrix_random_pairs(instances_sample, order=order)
        p_values_dict = dict(zip(["p_emb_emlap_vs_noscemus_alchymy", "p_emb_emlap_vs_noscemus_rest", "p_emb_emlap_vs_cc"], np.round(Ps.iloc[0].values[1:], 4)))
        overview_data.update(p_values_dict)
        return overview_data
    except Exception:
        return {}

In [65]:
overview_dict = obtain_overview_data("aurum.parquet")

In [66]:
overview_dict

{'emlap_instances_N': 11585,
 'noscemus_alchymia_instances_N': 5701,
 'noscemus_rest_instances_N': 14074,
 'cc_instances_N': 37216,
 'grela_la_instances_N': 68576,
 'emlap_instances_freq': 0.0017886322961067258,
 'noscemus_alchymia_instances_freq': 0.00039723522610355934,
 'noscemus_rest_instances_freq': 0.00011444724942576129,
 'cc_instances_freq': 0.00014607634505743193,
 'grela_la_instances_freq': 0.00017205367847959586,
 'emlap_vs_grela_la_rel_freq': 10.39578061865642,
 'p_emb_emlap_vs_noscemus_alchymy': np.float64(0.6334),
 'p_emb_emlap_vs_noscemus_rest': np.float64(0.4101),
 'p_emb_emlap_vs_cc': np.float64(0.0)}

In [68]:
overview_data_series = lexeme_df["instance_fname"].apply(obtain_overview_data)

/home/jupyter-vojta/notebooks/labyrinth/venv_torch_nlp/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/jupyter-vojta/notebooks/labyrinth/venv_torch_nlp/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/jupyter-vojta/notebooks/labyrinth/venv_torch_nlp/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **k

In [69]:
overview_data_series.to_json("../data/overview_data_series.json")

In [70]:
overview_data_series.to_pickle("../data/overview_data_series.pickle")


In [71]:
overview_data_series[0].keys()

dict_keys(['emlap_instances_N', 'noscemus_alchymia_instances_N', 'noscemus_rest_instances_N', 'cc_instances_N', 'grela_la_instances_N', 'emlap_instances_freq', 'noscemus_alchymia_instances_freq', 'noscemus_rest_instances_freq', 'cc_instances_freq', 'grela_la_instances_freq', 'emlap_vs_grela_la_rel_freq', 'p_emb_emlap_vs_noscemus_alchymy', 'p_emb_emlap_vs_noscemus_rest', 'p_emb_emlap_vs_cc'])

In [72]:
for key in overview_data_series[0].keys():
    lexeme_df[key] = overview_data_series.apply(lambda x: x.get(key, 0))

In [74]:
# conservative under Google’s 50k hard limit
MAX_CELL = 48000

def _clip(s: str, n: int = MAX_CELL) -> str:
    return s if len(s) <= n else (s[:n-1] + "…")

def _stringify(v) -> str:
    """Controlled stringification so lists/dicts don't explode."""
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return ""
    if isinstance(v, (list, dict)):
        # compact JSON, unicode-friendly
        return json.dumps(v, ensure_ascii=False, separators=(',', ':'))
    return str(v)

def prepare_for_sheets(df: pd.DataFrame, *, drop_heavy: bool = False) -> pd.DataFrame:
    """
    Make a sheets-safe copy:
      - optional: drop very heavy/nested cols
      - stringify lists/dicts in a compact way
      - clip any cell > MAX_CELL
    """
    if drop_heavy:
        df = df.drop(columns=[
            # keep this list lean; tweak as you like
            "senses_flat", "Senses", "Translations", "EntryLevelTranslations"
        ], errors="ignore").copy()
    else:
        df = df.copy()

    # stringify then clip per-cell
    df = df.map(_stringify)
    df = df.map(lambda s: _clip(s, MAX_CELL))
    return df

# --- example usage ---
# keep everything but serialize/clip:
df_for_sheet = prepare_for_sheets(lexeme_df, drop_heavy=False)

In [75]:
set_with_dataframe(ruland_gs.add_worksheet("ruland_emlap_instances_2025-12-24", 1,1), df_for_sheet.sort_values("emlap_instances_N", ascending=False))

In [76]:
lexeme_df.to_json("../data/large_files/ruland-emlap-grela.json")

In [77]:
lexeme_df.to_parquet("../data/large_files/ruland-emlap-grela.parquet")